In [1]:
using Distributions
using Roots

In [20]:
N = 100
R = 100
S = N * R
T = 100
simnum = 20
alpha = -0.01
a = 2
b = 2;

In [3]:
beta = Beta(a, b)  
u = rand(beta, N*T)
u = reshape(u, (N, T)) - 0.5;

In [14]:
function f(alpha, N, u_t)
    function f1(equi)
        equi - sum([1/(1 + exp(i + alpha*N*(1-2*equi))) for i in u_t])/N
    end
end

f (generic function with 1 method)

In [21]:
result = zeros(T)
box = zeros((simnum, T))
for n in 1:simnum
    srand(n)
    beta = Beta(a, b)  
    u = rand(beta, N*T)
    u = reshape(u, (N, T)) - 0.5
    for t in 1:T
        u_t = u[:, t]
        result[t] = fzero(f(alpha, N, u_t), 10e-5, 1 - 10e-5)
    end
    box[n, :] = result
end

In [22]:
# alphaが小さい方が大きくぶれるね。（当たり前）
box[1, :]

100-element Array{Float64,1}:
 0.504883
 0.510071
 0.495492
 0.508687
 0.502168
 0.505009
 0.502862
 0.499687
 0.496815
 0.500081
 0.502337
 0.500221
 0.499474
 ⋮       
 0.50781 
 0.49754 
 0.503124
 0.490396
 0.508746
 0.494697
 0.49261 
 0.503829
 0.498591
 0.503703
 0.504784
 0.504223

上で求めた均衡を元に、選択確率を計算し、データをシミュレート